In [8]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
import re
import time

In [9]:
# Cria a lista que armazena todos os links encontrados (/wiki/) e inicializa ela com um item nulo (página inicial)
listaSitesParaEntrar = []
listaSitesParaEntrar.append("")

listaSitesTitulos = []

# Cria a lista que armazena todos os sites que já foram visitados para coleta de links
listaSitesVisitados = []

# Cria a lista que armazena todos os sites já coletados de pessoas
listaPessoas = []

# Esvazia o arquivo pessoas.txt
with open("pessoas.txt", "w") as f:
    pass

In [10]:
# Função para transformar a url em BeautifulSoup
def busca(url):
    html = urlopen(Request(url, headers={
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }))
    site = html.read()
    return BeautifulSoup(site)

def salva(url, resposta):
    with open(f"./{url}.html", "w", encoding="utf-8") as arquivo:
        conteudo = resposta.decode("utf-8")
        arquivo.write(conteudo)


In [11]:
# Função que devolve todos os links relevantes (/wiki/) encontrados em uma url enviada
def pegaTodosLinks(url):
    print(f"========== PEGANDO OS LINKS DA PAGINA {url} ==========")
    pagina = busca(url)
    return pagina.find_all("a", href=re.compile('^(/wiki/)((?!:)(?<!Wikip)(?<!L%C3%ADngua).)*$'))

In [12]:
# Função que faz tudo que precisa fazer com um site
def visitaUmSite(site):
    # Adiciona esse site à lista de visitados
    listaSitesVisitados.append(site)
    todos_links = pegaTodosLinks(site)
    contador = 0
    for link in todos_links:
        if len(listaPessoas) > 1000:
            break

        # Se esse link já está na lista de espera pra visita ou já foi visitado, pula para o próximo
        if link["href"] in listaSitesParaEntrar or ("https://pt.wikipedia.org" + link["href"]) in listaSitesVisitados:
            continue

        listaSitesParaEntrar.append(link["href"])
        
        # Imprime o link e o número dele nessa página
        contador += 1
        print(f"{contador}. ", end = "")
        print(link["href"])

        #Obtém a infobox
        url = "https://pt.wikipedia.org" + link["href"]
        
        pagina = urlopen(Request(url, headers={
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }))
        resposta = pagina.read()
        
        soup = BeautifulSoup(resposta, "html.parser")
        box = soup.find("table", {"class": "infobox"})
        titulo = soup.find("title")
    
        if box is None or len(box) == 0:
            print("SEM INFOBOX")
        else:
            # Obtém os campos da infobox, tentando as 4 maneiras que consegui encontrar de disposição das tags
            tags = []
            for tag in box.find_all("th", class_="infobox-label"):
                tags.append(tag.get_text(strip=True))
            if len(tags) == 0:
                for tag in box.find_all("td", {"scope": "row"}):
                    tags.append(tag.get_text(strip=True))
            if len(tags) == 0:
                for tag in box.find_all("th", {"style": "text-align: left;"}):
                    tags.append(tag.get_text(strip=True))
            if len(tags) == 0:
                for tag in box.find_all("th"):
                    tags.append(tag.get_text(strip=True))
            print(tags)

            
            # Checa se "Nascimento" ou "Data de nascimento" está presente nas tags (ainda não encontrei uma terceira alternativa)
            if "Nascimento" in tags or "Data de nascimento" in tags:
                if titulo not in listaPessoas:
                    # Adiciona à lista de pessoas encontradas e ao arquivo pessoas.txt
                    print("+++++ É UMA PESSOA! +++++")
                    titulo = soup.find("span", {"class": "mw-page-title-main"})
                    titulo = titulo.text
                    
                    salva(link["href"].replace('.', ''), resposta)
                    with open("pessoas.txt", "a") as arquivo:
                        arquivo.write(link["href"])
                        arquivo.write("\n")
                    listaPessoas.append(titulo)
        print("Numero de Pessoas Encontradas: ", len(listaPessoas), "/ 1000")
        print()
    
        time.sleep(1)

    print(f"{len(todos_links)} links encontrados\n\n\n")
    

In [13]:
# Função que visita os x primeiros sites da lista de sites para entrar

def programa():
    while len(listaPessoas) < 1000:
        # Retira o item no topo da lista de sites para entrar e envia como link completo para a função visitaUmSite
        visitaUmSite("https://pt.wikipedia.org" + listaSitesParaEntrar.pop(0))

In [ ]:
# Roda o programa, retomando da pagina seguinte à última interrupção, ou começando do zero caso a segunda célula seja rodada
programa()

========== PEGANDO OS LINKS DA PAGINA https://pt.wikipedia.org/wiki/Augusto ==========
1. /wiki/Augusto_(desambigua%C3%A7%C3%A3o)
SEM INFOBOX
Numero de Pessoas Encontradas:  655 / 1000

2. /wiki/Augusto_de_Prima_Porta
['Autor', 'Data', 'Localização']
Numero de Pessoas Encontradas:  655 / 1000

3. /wiki/Imperador_romano
['Período histórico', 'Brasão']
Numero de Pessoas Encontradas:  655 / 1000

4. /wiki/16_de_janeiro
[]
Numero de Pessoas Encontradas:  655 / 1000

5. /wiki/14
[]
Numero de Pessoas Encontradas:  655 / 1000

6. /wiki/J%C3%BAlio_C%C3%A9sar
['Período', 'Período', 'Antecessor(a)', 'Sucessor(a)', 'Período', 'Antecessor(a)', 'Sucessor(a)', 'Período', 'Antecessor(a)', 'Sucessor(a)', 'Período', 'Antecessor(a)', 'Sucessor(a)', 'Nome completo', 'Nascimento', 'Morte', 'Nacionalidade', 'Progenitores', 'Cônjuge', 'Filhos(as)', 'Partido', 'Religião', 'Serviço/ramo', 'Graduação', 'Conflitos']
+++++ É UMA PESSOA! +++++
Numero de Pessoas Encontradas:  656 / 1000

7. /wiki/Tib%C3%A9rio
['Re

: 